# Kinematic Trajectory Optimization Around Shelf Exterior with the Kuka Iiwa

In this exercise you'll build on the examples in [Chapter 6](https://manipulation.csail.mit.edu/trajectories.html#section2) and better understand how the state of your robot's environment affects the techniques you use when formulating and solving your optimization problem. Specifically, we'll look at the effects of operating at the boundaries of the Iiwa's kinematic workspace and with complex collision constraints applied as you move its gripper from the upper RH side outside of the shelf to the bottom LH side outside of the shelf.

In [1]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    BsplineTrajectory,
    Context,
    DiagramBuilder,
    KinematicTrajectoryOptimization,
    MathematicalProgram,
    MathematicalProgramResult,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MinimumDistanceLowerBoundConstraint,
    MultibodyPlant,
    Parser,
    PositionConstraint,
    Rgba,
    RigidTransform,
    Role,
    Solve,
    Sphere,
    StartMeshcat,
)

from manipulation.meshcat_utils import PublishPositionTrajectory
from manipulation.scenarios import AddIiwa, AddWsg
from manipulation.utils import ConfigureParser

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


### Setup

In [3]:
# Clear your visualizer
meshcat.Delete()

In [4]:
# TODO: create your diagram builder
builder = DiagramBuilder()

# TODO: create your plant and scene graph with a time step of 0.001
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

# TODO: add an Iiwa robot to your plant and set collision model to be "with_box_collision"
iiwa = AddIiwa(plant, collision_model="with_box_collision")

# TODO: add the WSG gripper to your iiwa, set roll to 0.0 and weld it
wsg = AddWsg(plant, iiwa, welded=True, roll=0.0)

In [5]:
# TODO: visualize the start and end positions using a sphere
# Hint: you'll want to use meshcat's SetObject and SetTransform

X_WStart = RigidTransform([0.65, -0.5, 0.68])
X_WGoal = RigidTransform([0.6, 0.5, 0.15])

# Create and position the start sphere (red)
meshcat.SetObject("iiwa_start", Sphere(0.05), Rgba(1, 0, 0, 1))
meshcat.SetTransform("iiwa_start", X_WStart)

# Create and position the goal sphere (blue)
meshcat.SetObject("iiwa_goal", Sphere(0.05), Rgba(0, 0, 1, 1))
meshcat.SetTransform("iiwa_goal", X_WGoal)

In [6]:
# Using the parser to set up the shelves
parser = Parser(plant)
ConfigureParser(parser)
shelf = parser.AddModelsFromUrl("package://manipulation/shelves.sdf")[0]

# Add different objects to the shelves
cracker_box = parser.AddModelsFromUrl(
    "package://manipulation/hydro/003_cracker_box.sdf"
)[0]
sugar_box = parser.AddModelsFromUrl("package://manipulation/hydro/004_sugar_box.sdf")[0]
cracker_height = 0.065800 / 2.0
sugar_height = 0.039100 / 2.0

mustard_bottle = parser.AddModelsFromUrl(
    "package://manipulation/hydro/006_mustard_bottle.sdf"
)[0]

# Weld the shelf to the world
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("shelves_body", shelf),
    RigidTransform([0.6, 0, 0.4]),
)

# Weld objects to shelves
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_link_cracker", cracker_box),
    RigidTransform([0.6, 0, 0.53915 + cracker_height]),
)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_link_mustard", mustard_bottle),
    RigidTransform([0.6, 0.0, 0.27685]),
)
plant.WeldFrames(
    plant.world_frame(),
    plant.GetFrameByName("base_link_sugar", sugar_box),
    RigidTransform([0.56, 0.0, 0.016 + sugar_height]),
)

plant.Finalize()

In [7]:
# Setting up our meshcat and collision visualizers
visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(role=Role.kIllustration),
)
collision_visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(
        prefix="collision", role=Role.kProximity, visible_by_default=False
    ),
)

In [8]:
# TODO: set up your diagram, context, and plant_context
diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

In [9]:
# Saving the number of joints in the Iiwa, its default positon, and the end effector frame
num_q = plant.num_positions()
q0 = plant.GetPositions(plant_context)
gripper_frame = plant.GetFrameByName("body", wsg)

### Formulating our trajectory optimization

We'll begin by implementing a reusable trajectory optimization function.


We highly encourage reviewing the [Drake documentation]((https://drake.mit.edu/doxygen_cxx/classdrake_1_1planning_1_1trajectory__optimization_1_1_kinematic_trajectory_optimization.html)) for the imports provided before proceeding. None of the constraints and costs you will design below rely heavily on Numpy functionality, we're using Drake's mathematical program and optimization machinery to define and solve our formulation.

In [10]:
# TODO: create a kinematic trajectory optimization object with 15 control points
trajopt = KinematicTrajectoryOptimization(num_q, 15)
prog = trajopt.get_mutable_prog()

In [11]:
def formulate_trajopt(
    trajopt: KinematicTrajectoryOptimization, prog: MathematicalProgram
) -> None:
    # Create PositionConstraints for start and goal positions
    # Use [0, 0.1, 0] as the position of the gripping point with respect to the gripper frame
    p_BQ = np.array([0, 0.1, 0]).reshape(3, 1)  # Position of gripping point relative to gripper frame
    
    # Start position constraint
    start_pos = X_WStart.translation().reshape(3, 1)
    start_constraint = PositionConstraint(
        plant=plant,
        frameA=plant.world_frame(),
        p_AQ_lower=start_pos,
        p_AQ_upper=start_pos,
        frameB=gripper_frame,
        p_BQ=p_BQ,
        plant_context=plant_context
    )
    
    # Goal position constraint  
    goal_pos = X_WGoal.translation().reshape(3, 1)
    goal_constraint = PositionConstraint(
        plant=plant,
        frameA=plant.world_frame(),
        p_AQ_lower=goal_pos,
        p_AQ_upper=goal_pos,
        frameB=gripper_frame,
        p_BQ=p_BQ,
        plant_context=plant_context
    )
    
    # Add both constraints as PathPositionConstraints
    # Start constraint at s=0 (beginning of path)
    trajopt.AddPathPositionConstraint(start_constraint, s=0.0)
    # Goal constraint at s=1 (end of path)
    trajopt.AddPathPositionConstraint(goal_constraint, s=1.0)
    
    # Add position and velocity bounds for hardware limitations
    trajopt.AddPositionBounds(
        lb=plant.GetPositionLowerLimits().reshape(-1, 1),
        ub=plant.GetPositionUpperLimits().reshape(-1, 1)
    )
    trajopt.AddVelocityBounds(
        lb=plant.GetVelocityLowerLimits().reshape(-1, 1),
        ub=plant.GetVelocityUpperLimits().reshape(-1, 1)
    )
    
    # Add path velocity constraints at beginning and end (stationary)
    # Zero velocity at start (s=0)
    zero_velocity = np.zeros(plant.num_velocities()).reshape(-1, 1)
    trajopt.AddPathVelocityConstraint(
        lb=zero_velocity,
        ub=zero_velocity,
        s=0.0
    )
    # Zero velocity at end (s=1)  
    trajopt.AddPathVelocityConstraint(
        lb=zero_velocity,
        ub=zero_velocity,
        s=1.0
    )
    
    # Constrain duration between 0.5 and 5 seconds
    trajopt.AddDurationConstraint(lb=0.5, ub=5.0)
    
    # Add cost on duration and path length with magnitude 1.0
    trajopt.AddDurationCost(weight=1.0)
    trajopt.AddPathLengthCost(weight=1.0)
    
    # Add cost to encourage solution close to q0 at start and end
    # This "normalizes" the behavior by preferring configurations close to the default
    # Use quadratic cost on the control points
    control_points = trajopt.control_points()
    
    # Cost for staying close to q0 at the beginning (first control point)
    prog.AddQuadraticCost(
        Q=np.eye(plant.num_positions()),
        b=-2 * q0,
        c=np.dot(q0, q0),
        vars=control_points[:, 0]
    )
    
    # Cost for staying close to q0 at the end (last control point)
    prog.AddQuadraticCost(
        Q=np.eye(plant.num_positions()),
        b=-2 * q0,
        c=np.dot(q0, q0),
        vars=control_points[:, -1]
    )

In [12]:
formulate_trajopt(trajopt, prog)

### "Karate chop" formulation

In [13]:
def solve_prog(prog: MathematicalProgram) -> MathematicalProgramResult:
    result = Solve(prog)
    if not result.is_success():
        print("Trajectory optimization failed!")
        print(result.get_solver_id().name())
        if hasattr(result, "get_constraint_violations"):
            violations = result.get_constraint_violations()
            print(f"Constraint violations: {violations}")
        print(f"Solution result: {result.get_solution_result()}")

    return result


result = solve_prog(prog)

In [14]:
# This function will publish your trajectory on meshcat so you can see it
def push_traj_to_meshcat(
    traj: BsplineTrajectory,
    plant: MultibodyPlant,
    context: Context,
    visualizer: MeshcatVisualizer,
) -> None:
    PublishPositionTrajectory(traj, context, plant, visualizer)
    collision_visualizer.ForcedPublish(
        collision_visualizer.GetMyContextFromRoot(context)
    )

If your trajectory optimziation formulation is correct, the above solve will run. But what do you see in the visualizer?

You'll see the "karate chop", where the Iiwa ignores collisions with its environment and takes a straight path to the goal beside the bottom shelf. This is because we haven't added collision constraints to our trajectory optimization problem yet. We'll do that next.

In [15]:
push_traj_to_meshcat(trajopt.ReconstructTrajectory(result), plant, context, visualizer)
karate_chop_traj = trajopt.ReconstructTrajectory(result)

### Collision constraint formulation

With the addition of collision constraints, you'll this trajectory optimization fails to converge. If you visualize this trajectory on the Kuka Iiwa, you'll see a semi-reasonable trajectory is produced, but it does not strictly follow the constraints we established above.

In [16]:
def add_collision_constraints(
    trajopt: KinematicTrajectoryOptimization,
    plant: MultibodyPlant,
    plant_context: Context,
    bound: float = 0.005,
    influence_dist: float = 0.1,
) -> None:
    # Create a collision constraint using MinimumDistanceLowerBoundConstraint
    collision_constraint = MinimumDistanceLowerBoundConstraint(
        plant=plant,
        bound=bound,
        plant_context=plant_context,
        influence_distance_offset=influence_dist
    )

    evaluate_at_s = np.linspace(0, 1, 25)  # discretize your path parameter s

    # Add the collision constraint as a PathPositionConstraint along your discretization
    for s in evaluate_at_s:
        trajopt.AddPathPositionConstraint(collision_constraint, s=s)

As you'll see, this solve will not converge. We'll explore how to fix that in the next section. Before moving on, consider, how might we point our solver in the right direction? What is our initial guess right now?

In [17]:
# Now we add our intial guess, constraints, solve, and visualize the result
trajopt.SetInitialGuess(karate_chop_traj)
add_collision_constraints(trajopt, plant, plant_context)
result = solve_prog(prog)
push_traj_to_meshcat(trajopt.ReconstructTrajectory(result), plant, context, visualizer)

Trajectory optimization failed!
SNOPT
Solution result: SolutionResult.kIterationLimit


### RRT initial guess formulation

To find a solution that strictly satisfies our formulation, we need a better initial guess than the "karate chop". In the cell below, we provide a trajectory for this warmstart that generally moves the iiwa from the upper RH corner outside the shelf to the bottom LH corner outside the sehfl, but does so suboptimally. This sort of trajectory could be produced by an RRT solve, which (as we saw in class) oftentimes results in the constraint being satisfied, but with the suboptimal "RRT dance". Using it as an initial guess will bias our solver towards trajectories that don't collide with the shelves or objects, while smoothing out the inaccuracies of the coarse guess. 

In [18]:
# Construct our RRT-like initial guess
rrt_traj = np.array(
    [
        [
            -7.45689077e-01,
            6.37049031e-01,
            7.83693744e-03,
            -5.75270941e-01,
            1.65994177e-01,
            8.36982962e-01,
            -5.02864623e-06,
        ],
        [
            1.78124761e00,
            -1.17381807e00,
            -1.25543656e00,
            -7.61433526e-01,
            -4.40644608e-01,
            7.56665825e-01,
            -3.33635499e-08,
        ],
        [
            -5.39334528e-01,
            4.25779369e-01,
            -3.37934459e-01,
            -7.82920056e-01,
            -1.62384447e-01,
            1.02508682e00,
            -6.23967215e-07,
        ],
        [
            1.26694134e00,
            -9.82648441e-01,
            -1.36863303e00,
            -1.47898807e00,
            -1.64804135e00,
            -4.96802200e-01,
            -4.52591206e-07,
        ],
        [
            -3.18591057e-01,
            7.35840857e-03,
            -3.16430919e-01,
            -8.61990620e-01,
            -1.76379301e-01,
            1.29448226e00,
            0.00000000e00,
        ],
        [
            -3.18591057e-01,
            7.35840857e-03,
            -3.16430919e-01,
            -8.61990620e-01,
            -1.76379301e-01,
            1.29448226e00,
            0.00000000e00,
        ],
        [
            -1.75861608e-01,
            -1.26941659e-01,
            -1.92395479e-01,
            -9.15815426e-01,
            -1.15402782e-01,
            1.45028904e00,
            0.00000000e00,
        ],
        [
            -8.60034121e-02,
            -1.81136292e-01,
            -9.77674339e-02,
            -8.67526177e-01,
            -5.98366726e-02,
            1.43741637e00,
            0.00000000e00,
        ],
        [
            8.63055646e-02,
            1.43815437e-01,
            7.35054974e-02,
            -7.05844435e-01,
            3.98104529e-02,
            1.10015563e00,
            0.00000000e00,
        ],
        [
            2.78261972e-01,
            2.59568266e-01,
            2.00653598e-01,
            -6.41343526e-01,
            1.02444314e-01,
            9.67662265e-01,
            -5.84275288e-07,
        ],
        [
            5.86803636e-01,
            4.86779729e-01,
            1.95594196e-01,
            -4.74908330e-01,
            1.03898852e-04,
            7.05472718e-01,
            -4.20908672e-06,
        ],
        [
            8.25791815e-01,
            7.44026062e-01,
            -6.95064740e-02,
            -5.64983331e-01,
            -1.92809108e-01,
            8.22901363e-01,
            -2.10880759e-07,
        ],
        [
            8.50372520e-01,
            9.51203006e-01,
            -1.33958217e-01,
            -7.45635034e-01,
            -2.50462773e-01,
            9.57120899e-01,
            6.69304817e-06,
        ],
        [
            8.49834320e-01,
            1.07646661e00,
            -1.50533276e-01,
            -7.42544742e-01,
            -2.13626887e-01,
            9.11104644e-01,
            4.90654359e-07,
        ],
        [
            8.46290031e-01,
            1.05195103e00,
            -1.13954237e-01,
            -8.71740931e-01,
            -2.39890686e-01,
            1.02117671e00,
            -1.39408976e-07,
        ],
    ]
)

# Cast it to a BSplineTrajectory type
traj_warmstart = BsplineTrajectory(basis=trajopt.basis(), control_points=rrt_traj.T)

In [19]:
# TODO: set the initial guess
trajopt.SetInitialGuess(traj_warmstart)

# Now we solve and visualize the result
result = solve_prog(prog)
push_traj_to_meshcat(trajopt.ReconstructTrajectory(result), plant, context, visualizer)

## Verification

Please take a video of your trajectory optimization solution and upload it to Gradescope. A succesful video will navigate from the red dot to the green dot around the outside of the shelf and not collide with any geometry in the environment.